In [1]:
import pandas as pd
import numpy  as np
import glob
from datetime import date, timedelta, datetime
import re

In [2]:
## read the excel template, shared by dhruv

df_template= pd.read_excel(r'C:\Users\arvinmvn\Desktop\sssaudit\STEP_AUDIT_MANUAL_TEMPLATE.xlsx',)
df_template.head()

,Date,STEP_SPN_Merchant,STEP_SPN_Name,Email,STEP_SPN_Contact,Service,Service Provider,Service Name,SP Name,SP ID,Eligible Seller,VALID,Invalid Reason,Key,Requested,Count of Request in AJ,Created,Remarks,Sub Reason
0,2022-01-05,A102P3YYZV238V,Nikhil,seller@parnajewellery.com,9619908034,EBC,Evitamin Business Consulting Pvt Ltd(WEST),Enhanced Brand Content,Evitamin Business Consulting Pvt Ltd,860468c6-9e94-4627-8e89-23e58e944fff,1,1,Valid,A102P3YYZV238VEnhanced Brand Content,2,2,1,Created as per requirement,Bulk
1,2022-01-05,A102P3YYZV238V,Nikhil,seller@parnajewellery.com,9619908034,Cataloging,e-Vitamins(WEST),Cataloguing,Evitamin Business Consulting Pvt Ltd,860468c6-9e94-4627-8e89-23e58e944fff,1,1,Valid,A102P3YYZV238VCataloguing,2,2,1,Created as per requirement,Bulk
2,2022-01-05,A10PP0ZBB1O3UR,Anchor Socks,connect@anchorsocks.com,9324352886,EBC,REEFORMERS(WEST),Enhanced Brand Content,REEFORMERS,b3f2f78a-5443-48ea-8b17-28b337100ccc,1,1,Valid,A10PP0ZBB1O3UREnhanced Brand Content,2,2,1,Created as per requirement,Bulk
3,2022-01-05,A10PP0ZBB1O3UR,Anchor Socks,connect@anchorsocks.com,9324352886,Accounting,Lynex global (NORTH),Accounting,Lynex Global,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,1,1,Valid,A10PP0ZBB1O3URAccounting,2,2,1,Created as per requirement,Bulk
4,2022-01-05,A10S0CO01BSJXK,Tejas Maliwal,virat_maliwal@yahoo.co.in,9595633522,EBC,Reliq Eagle Consultancy Pvt Ltd(WEST),Enhanced Brand Content,Reliq Eagle Consultancy Pvt. Ltd.,18511f57-d981-4e9f-8cc0-1ab45bab360f,1,1,Valid,A10S0CO01BSJXKEnhanced Brand Content,2,1,1,Created as per requirement,Bulk


In [3]:
## take all the necessary sheets for further audit

## mapping files

Eligble_Seller= pd.read_excel(r'C:\Users\arvinmvn\Desktop\sssaudit\STEP_AUDIT_MANUAL_TEMPLATE.xlsx', sheet_name='Eligible Sellers')
Service_Map= pd.read_excel(r'C:\Users\arvinmvn\Desktop\sssaudit\STEP_AUDIT_MANUAL_TEMPLATE.xlsx', sheet_name='Service Map')
AJ_Sub_Service_Map=pd.read_excel(r'C:\Users\arvinmvn\Desktop\sssaudit\STEP_AUDIT_MANUAL_TEMPLATE.xlsx', sheet_name='AJ Sub Service Map')
SP_ID_Map=pd.read_excel(r'C:\Users\arvinmvn\Desktop\sssaudit\STEP_AUDIT_MANUAL_TEMPLATE.xlsx', sheet_name='SP ID Map')
SP_Name_Map=pd.read_excel(r'C:\Users\arvinmvn\Desktop\sssaudit\STEP_AUDIT_MANUAL_TEMPLATE.xlsx', sheet_name='SP Name Map')

## aj

aj=pd.read_excel(r'C:\Users\arvinmvn\Desktop\Step Automation Audit\Data\2022-03-14\AJ_latest_today.xlsx')


## already allocate files

already_alloc_aplus=pd.read_excel(r'C:\Users\arvinmvn\Desktop\sssaudit\Mappings\Leads Already Allocated - Aplus.xlsx')
already_alloc_main=pd.read_excel(r'C:\Users\arvinmvn\Desktop\sssaudit\Mappings\Leads Already Allocated - Main Services.xlsx')


In [4]:
## filter the aj file, take only requestsource as 'Referral' and requestsourcereftag as 'Step'

aj=aj[aj['requestsource']=='Referral']

aj=aj[aj['requestsourcereftag']=='STEP Program']


## rename Advertisement Optimization and Cataloging to match the service names for req_list df

aj['servicename']=aj['servicename'].replace('Advertising Optimization','Advertisement Optimization')
aj['servicename']=aj['servicename'].replace('Cataloguing','Cataloging')


aj.head()

,orderid,servicename,providercustomerid,sellercustomerid,marketplaceid,providerid,createdtimeinutc,followuptimeinutc,closebytimeinutc,selleremailid,...,ordercreationdate_new,servicename_derv,ordercreation_date_derv,createdtimeinutc_derv,workstartdateinutc_derv,workcompletiondate_derv,firstpitchindateinutc_derv,lastupdatedon_derv,start_date,end_date
0,0715cc1b-4a37-4680-a6ae-fd418fedc18b,Enhanced Brand Content,AAVK7011YROF1,A3L0FS1LQIVMDI,A21TJRUUN4KGV,93b03a01-a19d-4cea-903e-6ff16e9c4fe2,1643343191000,0,0,AAAAAAAAAQChFqlBs+1sM/tKBRJdrjiXKwAAAAAAAAAeeJ...,...,28-01-2022 04:13:11,Enhanced Brand Content,2022-01-27 20:13:11.0,2022-01-28 04:13:11.0,1970-01-01 00:00:00.0,1970-01-01 00:00:00.0,2022-01-30 16:58:45.0,2022-01-30 16:58:45.0,NaN,NaN
1,0a2cb689-442e-4165-873c-f952004dfea9,Imaging,A2S9DYN5JJGTCO,A2DUPWIEX3A4T4,A21TJRUUN4KGV,348d6610-bf8c-40e0-985e-4c913b7d1f78,1642097495000,0,0,AAAAAAAAAQDw/DhcvhvmCfXRCij1nrm8MwAAAAAAAACfJ9...,...,13-01-2022 18:11:35,Imaging,2022-01-13 10:11:35.0,2022-01-13 18:11:35.0,1970-01-01 00:00:00.0,1970-01-01 00:00:00.0,2022-01-20 12:19:27.0,2022-01-20 12:19:27.0,NaN,NaN
2,0b1a81c7-8f49-46d5-92e4-526ee23d9b63,Enhanced Brand Content,AV944QAHJ8H2J,ABCA7MJVQ3WQP,A21TJRUUN4KGV,860468c6-9e94-4627-8e89-23e58e944fff,1644820552000,0,0,AAAAAAAAAQAO2LBj6e8peNzVVLTsjE+9OAAAAAAAAAAsaE...,...,14-02-2022 06:35:52,Enhanced Brand Content,2022-02-13 22:35:52.0,2022-02-14 06:35:52.0,1970-01-01 00:00:00.0,1970-01-01 00:00:00.0,2022-02-14 07:13:49.0,2022-02-14 07:13:49.0,NaN,NaN
3,659eadcc-5ffb-4ed6-bc4a-6b76f645e7f3,Cataloging,A99ULOCIKOHMX,A3BO6AX1EO1L7B,A21TJRUUN4KGV,26ddd595-c3a2-413f-bec2-2de54264ede8,1643808614000,0,0,AAAAAAAAAQBWIrK1bZwVEtyK6PS38s08MwAAAAAAAACpRp...,...,02-02-2022 13:30:14,Cataloguing,2022-02-02 05:30:14.0,2022-02-02 13:30:14.0,1970-01-01 00:00:00.0,1970-01-01 00:00:00.0,2022-02-03 16:02:31.0,2022-02-03 16:02:31.0,NaN,NaN
4,3708310c-9379-440c-9250-45fd99e9e33c,Enhanced Brand Content,AV944QAHJ8H2J,A13A2GAKMT17NK,A21TJRUUN4KGV,860468c6-9e94-4627-8e89-23e58e944fff,1641995172000,0,0,AAAAAAAAAQBwbW/8t9RRme4/hc1fc0wwNwAAAAAAAADepM...,...,12-01-2022 13:46:12,Enhanced Brand Content,2022-01-12 05:46:12.0,2022-01-12 13:46:12.0,1970-01-01 00:00:00.0,1970-01-01 00:00:00.0,2022-01-13 08:22:15.0,2022-01-13 08:22:15.0,NaN,NaN


In [5]:
## read the column filtered pardot file for latest run

pardot= pd.read_excel(r'C:\Users\arvinmvn\Desktop\Step Automation Audit\Data\2022-03-14\col_filtered_download_file.xlsx')
len(pardot)                      

4031

In [6]:
#### Total STEP Requests Received

## add enhanced enhanced brand content for all the line items in the list (do a into 2), this will help to drive the total step requests

pardot_ebc=pardot.copy() ## assign the pardot file to pardot a+
pardot_ebc['Select one of the SPN services']='Enhanced Brand Content' ## replace all the column names with EBC

combine_list=[pardot,pardot_ebc]

pardot_v2 = pd.concat(combine_list)

len(pardot_v2)

print('Total STEP Requests Received is',len(pardot_v2))

Total STEP Requests Received is 8062


In [7]:
#### Valid  STEP Requests Received

## to check validity , we will evaluate from the following

# a.seller is eligible :- check with the seller mapping availble tor step and see if they are eligble
# b.has not already availed benefits:- check if they have aleardy taken the service using the already allocated leads 
# c. is interested :- take the requests who have not opted i dont want spn or aplus service
# d. not already been handled by the audit proces :-  check if this is already handle in audit
            

In [8]:
# Eligble sellers filtration (joint with service mappings on sellercustomerid)

seller_join= pd.merge(pardot_v2,Eligble_Seller, left_on="STEP_SPN_Merchant",right_on='encrypted_sellerid',how='left')

eligble_seller=seller_join[seller_join['encrypted_sellerid'].notna()]
ineligble_seller=seller_join[seller_join['encrypted_sellerid'].isna()]


print('Eligble Sellers Request:',len(eligble_seller))
print('Ineligble Sellers Request:',len(ineligble_seller))

Eligble Sellers Request: 7590
Ineligble Sellers Request: 472


In [9]:
## seller not interested in service list


not_interested=pd.concat([pardot[pardot['Select one of the services for Aplus']=='I do not want A+ cataloging'], 
                           pardot[pardot['Select one of the SPN services']=='I do not want SPN service']]).drop_duplicates()

print('sellers not interested in service:-',len(not_interested))
print('sellers interested in service:-',len(pardot)-len(not_interested))

sellers not interested in service:- 367
sellers interested in service:- 3664


In [10]:
## Invalid requests is the sum of the distinct list of eligble seller and  not interested

#eligble_seller[eligble_seller['encrypted_sellerid'].isna()]-> inelgible seller

invalid_step_requests= pd.concat([not_interested,ineligble_seller]).drop_duplicates()
print('Invalid STEP Requests Received :- ',len(invalid_step_requests))

Invalid STEP Requests Received :-  773


In [11]:
## Valid request is the subraction of total step request minus invalid requests

print('Valid STEP Requests Recieved :-' ,len(pardot_v2)-len(invalid_step_requests))

Valid STEP Requests Recieved :- 7289


In [12]:
## Step allocate Request is the total number of requests available in AJ

print('STEP Requests Allocated',len(aj))

STEP Requests Allocated 7022


In [13]:
## Step leads Allocated via automation


directory=r'C:\Users\arvinmvn\Desktop\Step Automation Audit\Data/'

upload_file_tracker=pd.DataFrame()

for filepath in glob.iglob(directory+'/*'):
    
    try:
        #print(filepath)

        d=pd.read_excel(filepath+'/Upload_file_tracker.xlsx')
        upload_file_tracker=upload_file_tracker.append(d)
            
    except:
        
        #print('error:', filepath)
        pass
 

upload_file_tracker=upload_file_tracker.drop_duplicates().reset_index(drop=True)

print('The Step Leads Allocated Via Bulk Automation',len(upload_file_tracker))

The Step Leads Allocated Via Bulk Automation 6256


In [14]:
## to find the total steps unallocated, let's dive deep on the requests on available in AJ and filter out the possible eliglble ones


# join aj and pardot_v2 on servicename and seller


not_found_aj_v1 =pd.merge(pardot_v2,aj, left_on=['STEP_SPN_Merchant','Select one of the SPN services'],right_on=['sellercustomerid','servicename'],how='left')

not_found_aj_v2=not_found_aj_v1[not_found_aj_v1['sellercustomerid'].isna()]

print('total seller line items not allocated in AJ',len(not_found_aj_v2))

## eligble sellers in the list


not_found_aj_v3=pd.merge(not_found_aj_v2,Eligble_Seller, left_on="STEP_SPN_Merchant",right_on='encrypted_sellerid',how='left')

not_found_aj_v3 = not_found_aj_v3[not_found_aj_v3['encrypted_sellerid'].notna()]

print('total eligble seller line items not allocated in AJ',len(not_found_aj_v3))


#already availed

## create amazon comments key and derive already availed number using key of sellerid and amazon comments

not_found_aj_v3['amazon comments']=''

for i in range (0,len(not_found_aj_v3['amazon comments'])):
    if not_found_aj_v3['Select one of the SPN services'].values[i]=='Enhanced Brand Content':
        
        not_found_aj_v3['amazon comments'].values[i]='STEP Offer - A+ Cataloguing - Up to 30 ASINs'
        
    if not_found_aj_v3['Select one of the SPN services'].values[i]=='Cataloguing':
        
        not_found_aj_v3['amazon comments'].values[i]='STEP Offer - Cataloguing - INR 3500 Credits' 
        
    if not_found_aj_v3['Select one of the SPN services'].values[i]=='Account Management':
        not_found_aj_v3['amazon comments'].values[i]='STEP Offer - Account Management - INR 3500 Credits' 
        
    if not_found_aj_v3['Select one of the SPN services'].values[i]=='Brand Videos':
        not_found_aj_v3['amazon comments'].values[i]='STEP Offer - Brand Videos - INR 3500 Credits'
        
    if not_found_aj_v3['Select one of the SPN services'].values[i]=='Imaging':
        not_found_aj_v3['amazon comments'].values[i]='STEP Offer - Imaging - INR 3500 Credits'  
        
    if not_found_aj_v3['Select one of the SPN services'].values[i]=='Advertisement Optimization':
        not_found_aj_v3['amazon comments'].values[i]='STEP Offer - Advertising Optimization - INR 3500 Credits' 
        
    if not_found_aj_v3['Select one of the SPN services'].values[i]=='Training':
        not_found_aj_v3['amazon comments'].values[i]='STEP Offer - Training - INR 3500 Credits'     
        
    if not_found_aj_v3['Select one of the SPN services'].values[i]=='Accounting':
        not_found_aj_v3['amazon comments'].values[i]='STEP Offer - Accounting - INR 3500 Credits'
        

not_found_aj_v4=pd.merge(not_found_aj_v3,aj,left_on=['STEP_SPN_Merchant','amazon comments'],right_on=['sellercustomerid','amazoncomments'],how='left')
not_found_aj_v4=not_found_aj_v4[not_found_aj_v4['amazoncomments_y'].isna()]

print('total seller with who have not already availed service keys',len(not_found_aj_v4))


## remove invalid MCIDs

not_found_aj_v5 = not_found_aj_v4[not_found_aj_v4['STEP_SPN_Merchant'].str.match('A')]
not_found_aj_v5_view = not_found_aj_v2[not_found_aj_v2['STEP_SPN_Merchant'].str.match('A')].drop_duplicates() ## for view

print('total seller items that starts with MCIDs as A',len(not_found_aj_v5_view))


## remove invalid phone nos

not_found_aj_v6=not_found_aj_v5[not_found_aj_v5['STEP_SPN_Contact'].str.len()==10]
not_found_aj_v6_view=not_found_aj_v2[not_found_aj_v2['STEP_SPN_Contact'].str.len()!=10]

print('total seller items with valid phone numbers',len(not_found_aj_v6_view))


# map the name from sp name list

not_found_aj_v6['Service Provider'] = not_found_aj_v6.iloc[:, 1:].bfill(axis=1).iloc[:,6].fillna('unknown')


## Remove the char inside the bracket and clean the Service Provider name
for i in range(0,len(not_found_aj_v6['Service Provider'])):
    not_found_aj_v6['Service Provider'].values[i]=re.sub("\(.*?\)","()",not_found_aj_v6['Service Provider'].values[i]).strip("()")

    
## Join the Service Provider with Provider Mapping and get provider id 

not_found_aj_v7=pd.merge(not_found_aj_v6,SP_ID_Map, left_on=['Select one of the SPN services','Service Provider'],right_on=['Service','SP Name'],how='left')

not_found_aj_v7=not_found_aj_v7[not_found_aj_v7['Provider_Id'].notna()]

print('total seller items with mapped provider ids from mapping file',len(not_found_aj_v7))

#not_found_aj_v7.to_excel(r"C:\Users\arvinmvn\Desktop\test.xlsx")

total seller line items not allocated in AJ 2440
total eligble seller line items not allocated in AJ 1968
total seller with who have not already availed service keys 1113
total seller items that starts with MCIDs as A 2048
total seller items with valid phone numbers 27


<ipython-input-14-51094ba4c4c6>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_found_aj_v6['Service Provider'] = not_found_aj_v6.iloc[:, 1:].bfill(axis=1).iloc[:,6].fillna('unknown')


total seller items with mapped provider ids from mapping file 843


In [15]:
## filter the required columns

not_found_aj_v8=not_found_aj_v7[['Last Activity At',
                                'Do you agree to all the terms and conditions?',
                               'STEP_SPN_Merchant',
                               'STEP_SPN_Name',
                               'Email',
                               'STEP_SPN_Contact',
                               'Select one of the SPN services',
                               'Service Provider',
                               'Provider_Id']]
                                
                                
len(not_found_aj_v8[not_found_aj_v8['Select one of the SPN services']!='Brand Videos'])               

460

In [16]:
#invalid provider/sub service list

invalid_provider =not_found_aj_v2.copy()

## create service name
invalid_provider['Service Provider'] = not_found_aj_v2.iloc[:, 1:].bfill(axis=1).iloc[:,6].fillna('unknown')


## Remove the char inside the bracket and clean the Service Provider name
for i in range(0,len(invalid_provider['Service Provider'])):
    invalid_provider['Service Provider'].values[i]=re.sub("\(.*?\)","()",invalid_provider['Service Provider'].values[i]).strip("()")

    
## Join the Service Provider with Provider Mapping and get provider id 

invalid_provider=pd.merge(invalid_provider,SP_ID_Map, left_on=['Select one of the SPN services','Service Provider'],right_on=['Service','SP Name'],how='left')

invalid_provider=invalid_provider[invalid_provider['Provider_Id'].isna()]

print('total seller items with mapped provider ids from mapping file',len(invalid_provider.drop_duplicates()))


total seller items with mapped provider ids from mapping file 804


In [30]:
## date

days_to_subtract=3
d = datetime.today() - timedelta(days=days_to_subtract)
d.strftime('%Y-%m-%d')


df = pd.DataFrame(columns=['Report Name', 
                           'Current Quarter',
                           'Date',
                           'Total STEP Requests Received',
                           'Valid STEP Requests Received',
                           'Invalid STEP Requests Received',
                           'Ineligible Seller',
                           'Service not chosen',
                           'Steps Request allocated',
                           'Allocated via Automation',
                           'STEP Requests Unallocated', 
                           'Already Availed Benefits',
                           'Invalid Provider/Service/Sub Service',
                           #'Invalid Seller Info - Email, Location',
                           'Invalid Seller Info - Phone',
                           'Invalid Seller Info - MID Issue', 
                           'EBC Conflict',
                           'Reupload List'])

df = df.append({'Report Name': 'Step Program Audit Report',
                'Current Quarter':'Q1',
                'Total STEP Requests Received': len(pardot_v2),
                'Valid STEP Requests Received':len(pardot_v2)-len(invalid_step_requests),
                'Invalid STEP Requests Received':len(invalid_step_requests),
                'Ineligible Seller':len(ineligble_seller),
                'Service not chosen':len(not_interested),
                'Steps Request allocated': len(aj),
                'Allocated via Automation':len(upload_file_tracker),
                'STEP Requests Unallocated':len(not_found_aj_v2)-len(not_found_aj_v8),
                'Already Availed Benefits':len(not_found_aj_v4),
                'Invalid Provider/Service/Sub Service':len(invalid_provider),
                'EBC Conflict':len(ebc_conflict[conflict['STEP_SPN_Contact_x'].isna()]),
                #'Invalid Seller Info - Email, Location':'',
                'Invalid Seller Info - MID Issue':len(not_found_aj_v2)-len(not_found_aj_v5_view),
                'Invalid Seller Info - Phone':len(not_found_aj_v6_view),
                'Reupload List':len(not_found_aj_v8[not_found_aj_v8['Select one of the SPN services']!='Brand Videos'])  ,
                'Date': str(d.strftime('%Y-%m-%d'),)}, ignore_index=True)

# df_audit_flash.transpose()
df.transpose()
df_audit_report=df.transpose()
df_audit_report=df_audit_report.rename(columns={df_audit_report.columns[0]: 'Scale'})
df_audit_report['Description']=''
df_audit_report['Description'][0]='Name of the Program'
df_audit_report['Description'][1]='Business Quarter of the year'
df_audit_report['Description'][2]='Latest Report run date'
df_audit_report['Description'][3]='Overall Requests Recieved'
df_audit_report['Description'][4]='Total Valid Requests Recieved'
df_audit_report['Description'][5]='Invalid Requests Recieved (ineligble seller+ service not chosen)'
df_audit_report['Description'][6]='List of sellers who have applied for service but not part of STEP'
df_audit_report['Description'][7]='Sellers not interested in SPN service/A+ service',
df_audit_report['Description'][8]='Allocated list available from AJ'
df_audit_report['Description'][9]='Allocation via Automation(taken from total mappings)',
df_audit_report['Description'][10]='Unallocated Request',
df_audit_report['Description'][11]='Unallocated Request because of already availed benfits'
df_audit_report['Description'][12]='Unallocated Request because of incorrect details'
df_audit_report['Description'][13]='Unallocated Request because of incorrect phone numbers'
df_audit_report['Description'][14]='Unallocated Request because of MIDs'
df_audit_report['Description'][15]='Unallocated Request because of EBC'
df_audit_report['Description'][16]='Reupload list'
df_audit_report

,Scale,Description
Report Name,Step Program Audit Report,Name of the Program
Current Quarter,Q1,Business Quarter of the year
Date,2022-03-13,Latest Report run date
Total STEP Requests Received,8062,Overall Requests Recieved
Valid STEP Requests Received,7289,Total Valid Requests Recieved
Invalid STEP Requests Received,773,Invalid Requests Recieved (ineligble seller+ s...
Ineligible Seller,472,List of sellers who have applied for service b...
Service not chosen,367,(Sellers not interested in SPN service/A+ serv...
Steps Request allocated,7022,Allocated list available from AJ
Allocated via Automation,6256,(Allocation via Automation(taken from total ma...


In [ ]:
## Report template

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(r'C:\Users\arvinmvn\Desktop\sssaudit\auditreport.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_audit_report.to_excel(writer, sheet_name='Summary',header = True)
# df_all_leads_for_audit.to_excel(writer, sheet_name='Overall leads for audit')
# ineligible_leads_unique.to_excel(writer, sheet_name='Ineligible Seller')
# df_clean_phone_list[df_clean_phone_list['Phone_len']!=10].to_excel(writer, sheet_name='Invalid Phone No list')
# mid_issue.to_excel(writer, sheet_name='mid_issue list')
# df_form_export.to_excel(writer, sheet_name='pardot')
# df_aj_latest.to_excel(writer, sheet_name='AJ_latest')


# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [27]:
## already availed



already_alloc=pd.concat([already_alloc_aplus,already_alloc_main]).drop_duplicates()

not_found_aj_v4=pd.merge(not_found_aj_v3,already_alloc,right_on=['Seller ID','Service Category'],left_on=['STEP_SPN_Merchant','Select one of the SPN services'],how='left')

not_found_aj_v4=not_found_aj_v4[not_found_aj_v4['Seller ID'].isna()]

print('total seller with who have not already availed service keys',len(not_found_aj_v4))

total seller with who have not already availed service keys 1117


In [ ]:
#already availed

## create amazon comments key and derive already availed number using key of sellerid and amazon comments

not_found_aj_v7['amazon comments']=''

for i in range (0,len(not_found_aj_v7['amazon comments'])):
    if not_found_aj_v7['Select one of the SPN services'].values[i]=='Enhanced Brand Content':
        
        not_found_aj_v7['amazon comments'].values[i]='STEP Offer - A+ Cataloguing - Up to 30 ASINs'
        
    if not_found_aj_v7['Select one of the SPN services'].values[i]=='Cataloging':
        
        not_found_aj_v7['amazon comments'].values[i]='STEP Offer - Cataloguing - INR 3500 Credits' 
        
    if not_found_aj_v7['Select one of the SPN services'].values[i]=='Account Management':
        not_found_aj_v7['amazon comments'].values[i]='STEP Offer - Account Management - INR 3500 Credits' 
        
    if not_found_aj_v7['Select one of the SPN services'].values[i]=='Brand Videos':
        not_found_aj_v7['amazon comments'].values[i]='STEP Offer - Brand Videos - INR 3500 Credits'
        
    if not_found_aj_v7['Select one of the SPN services'].values[i]=='Imaging':
        not_found_aj_v7['amazon comments'].values[i]='STEP Offer - Imaging - INR 3500 Credits'  
        
    if not_found_aj_v7['Select one of the SPN services'].values[i]=='Advertisement Optimization':
        not_found_aj_v7['amazon comments'].values[i]='STEP Offer - Advertising Optimization - INR 3500 Credits' 
        
    if not_found_aj_v7['Select one of the SPN services'].values[i]=='Training':
        not_found_aj_v7['amazon comments'].values[i]='STEP Offer - Training - INR 3500 Credits'     
        
    if not_found_aj_v7['Select one of the SPN services'].values[i]=='Accounting':
        not_found_aj_v7['amazon comments'].values[i]='STEP Offer - Accounting - INR 3500 Credits'
        

In [28]:
test=pd.merge(not_found_aj_v7,aj,left_on=['STEP_SPN_Merchant','amazon comments'],right_on=['sellercustomerid','amazoncomments'],how='left')
test_new=test[test['amazoncomments_y'].isna()]
len(test_new)

843

In [ ]:
test_new.to_excel(r"C:\Users\arvinmvn\Desktop\test.xlsx")

In [ ]:
not_found_aj_v3['amazon comments'].unique()

In [25]:
## count of ineligble seller list overall

directory=r'C:\Users\arvinmvn\Desktop\Step Automation Audit\Data/'

ebc=pd.DataFrame()


for filepath in glob.iglob(directory+'/*'):
    
    try:
       # print(filepath)
        ebc_file=glob.glob(filepath+'/*EBC_Videos_Raw.xlsx')
        for file in ebc_file:
            b=pd.read_excel(file)
            ebc=ebc.append(b)
        

        
    except:
        
        #print('error:', filepath)
        pass
 
ebc_result = ebc.drop_duplicates().reset_index(drop=True)


# ineligible_leads_unique.to_excel(r"C:\Users\arvinmvn\Desktop\Step Automation Audit\total_ineligible_leads.xlsx")
# provider_not_found.to_excel(r"C:\Users\arvinmvn\Desktop\Step Automation Audit\total_provider_not_found_leads.xlsx")
# upload_file_tracker.to_excel(r"C:\Users\arvinmvn\Desktop\Step Automation Audit\total_uploads_file_tracker.xlsx")
# mid_issue.to_excel(r"C:\Users\arvinmvn\Desktop\Step Automation Audit\total_mid_issue_leads.xlsx")

len(ebc)

0

In [29]:
## join the ebc appended df and the final file to find out the ebc conflicts

ebc_conflict=pd.merge(not_found_aj_v8,ebc,on=['Email'],how='left')
len(ebc_conflict[ebc_conflict['STEP_SPN_Contact_x'].isna()])

KeyError: 'Email'

In [ ]:
not_found_aj_v1[not_found_aj_v1['sellercustomerid'].isna()]

In [31]:
invalid_step_requests.to_excel(r"C:\Users\arvinmvn\Desktop\invalid.xlsx")